In [3]:
import json
import random
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.metrics import accuracy_score

model_name = "Qwen/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="cuda"
)

texts = [
    "Футбольный матч между сборными России и Германии завершился со счётом 2:1 в пользу хозяев.",
    "Блокчейн-платформа Ethereum перешла на энергоэффективный алгоритм PoS, снизив потребление энергии на 99.95%.",
    "Постный борщ с фасолью и свеклой: вкусный и полезный вариант для вегетарианцев.",
    "Совет Безопасности ООН провёл экстренное заседание по ситуации на Ближнем Востоке.",
    "Исследование Университета Гарварда: регулярные прогулки снижают риск сердечно-сосудистых заболеваний на 40%.",
]
labels = [0, 1, 2, 3, 4]  # 5 классов: спорт, технологии, кулинария, политика, здоровье

# Настройки
classes = ["sport", "technology", "culinary", "politics", "health"]
class_indices = {cls: i for i, cls in enumerate(classes)}
num_classes = len(classes)

# Функция создания системного и пользовательского сообщения
def create_messages(text, examples=None):
    system_msg = "Вы - классификатор текстов. Назначьте каждому классу 0 или 1 в зависимости от текста."
    user_msg = f"Классифицируйте следующий текст в один из классов: {', '.join(classes)}.\n\nТекст: {text}"
    if examples:
        user_msg += "\n\nПримеры:"
        for example in examples:
            example_text = example["text"]
            example_label = example["label"]
            user_msg += f"\nТекст: {example_text}\nКлассификация: {example_label}"
    user_msg += "\n\nВыведите JSON-объект с ключами как названия классов и значениями 0 или 1, причем 1 должно быть ровно одно значение, остальные 0."
    return [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_msg}
    ]

# Генерация ответа модели с разделением на "thinking" и "content"
def generate_response(text, examples=None, temperature=0.7, enable_thinking=False):
    messages = create_messages(text, examples)
    text_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=enable_thinking  # Используем режим мышления
    )
    inputs = tokenizer(text_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=32768,
        temperature=temperature,
        do_sample=True
    )
    # output_text = tokenizer.decode(outputs[0], skip_special_tokens=False)  # Сохраняем спецсимволы
    # print("Полный ответ модели:\n", output_text)  # Выводим начало для понимания

    output_ids = outputs[0][len(inputs.input_ids[0]):].tolist() 

    # parsing thinking content
    try:
        # rindex finding 151668 (</think>)
        index = len(output_ids) - output_ids[::-1].index(151668)
    except ValueError:
        index = 0

    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

    # print("Thinking content:\n", thinking_content)  # Печатаем часть мышления
    # print("Content:\n", content)  # Печатаем часть ответа
    # print("-" * 50)

    return content


def parse_output(output, use_probabilities=False):
    try:
        # Удаление markdown-разметки
        output = output.replace("```json", "").replace("```", "").strip()
        data = json.loads(output)
        
        # Если data - список, берем первый элемент
        if isinstance(data, list):
            if len(data) > 0:
                data = data[0]
            else:
                raise ValueError("Пустой список в JSON-ответе")
                
        # Проверяем, что data - словарь
        if not isinstance(data, dict):
            raise ValueError(f"Ожидался словарь, получили {type(data)}")
            
        result = {}
        for cls in classes:
            # Используем русские ключи
            result[cls] = int(data.get(cls, 0))
        print("Предсказанные значения:", result)
        return result
    except (json.JSONDecodeError, ValueError):
        print("Ошибка парсинга. Используем нулевой вектор.")
        return {cls: 0 for cls in classes}

# Zero-shot классификация
def zero_shot_classify(text):
    response = generate_response(text)
    return parse_output(response)

# One-shot классификация
def one_shot_classify(text, examples):
    response = generate_response(text, examples=examples)
    return parse_output(response)

# Few-shot классификация
def few_shot_classify(text, examples):
    response = generate_response(text, examples=examples)
    return parse_output(response)

# Вероятностная классификация с усреднением
def probabilistic_classification(text, num_iterations=5):
    probabilities = {cls: 0.0 for cls in classes}
    temperatures = [0.5, 0.7, 1.0, 1.2, 1.5]
    
    print("Начинаем вероятностную классификацию для текста:", text[:30] + "...")
    for i in range(num_iterations):
        temp = random.choice(temperatures)
        print(f"Итерация {i+1}, температура: {temp}")
        response = generate_response(text, temperature=temp)
        pred = parse_output(response, use_probabilities=True)
        for cls in classes:
            probabilities[cls] += pred.get(cls, 0)
    
    # Нормализация
    total = sum(probabilities.values())
    if total > 0:
        probabilities = {k: v/total for k, v in probabilities.items()}
    else:
        probabilities = {k: 1/num_classes for k in classes}
    
    print("Нормализованные вероятности:", probabilities)
    print("-" * 50)
    return probabilities

# Оценка точности
def evaluate_classifiers():
    true_labels = []
    zero_shot_preds = []
    one_shot_preds = []
    few_shot_preds = []
    prob_preds = []

    # Примеры для one-shot и few-shot
    one_shot_examples = [
        {"text": "Баскетболисты ЦСКА одержали уверенную победу над испанским «Реалом» в Евролиге.", "label": {"sport": 1, "technology": 0, "culinary": 0, "politics": 0, "health": 0}},
        {"text": "Квантовые компьютеры MIT научились решать задачи за секунды, которые обычным системам требовались дни.", "label": {"sport": 0, "technology": 1, "culinary": 0, "politics": 0, "health": 0}},
        {"text": "Как приготовить домашнюю лапшу рамен за 30 минут — пошаговый гид с фотографиями.", "label": {"sport": 0, "technology": 0, "culinary": 1, "politics": 0, "health": 0}},
        {"text": "Выборы в Европарламент 2024: рост популярности экологических партий в Германии и Франции.", "label": {"sport": 0, "technology": 0, "culinary": 0, "politics": 1, "health": 0}},
        {"text": "Психологические практики: эффективные методы борьбы с тревожностью без лекарств.", "label": {"sport": 0, "technology": 0, "culinary": 0, "politics": 0, "health": 1}}
    ]
    few_shot_examples = one_shot_examples  # Для простоты используем те же примеры

    for i, text in enumerate(texts):
        true_label = labels[i]
        true_labels.append(true_label)

        print(f"\nТекст {i+1}: {text}")
        print("True Label:", classes[true_label])

        # Zero-shot
        print("Zero-shot:")
        zero_pred = zero_shot_classify(text)
        zero_shot_preds.append(max(zero_pred, key=zero_pred.get))

        # One-shot
        print("One-shot:")
        one_pred = one_shot_classify(text, one_shot_examples[:1])
        one_shot_preds.append(max(one_pred, key=one_pred.get))

        # Few-shot
        print("Few-shot:")
        few_pred = few_shot_classify(text, few_shot_examples)
        few_shot_preds.append(max(few_pred, key=few_pred.get))

        # Вероятностная классификация
        prob_pred = probabilistic_classification(text)
        prob_preds.append(max(prob_pred, key=prob_pred.get))

    # Вычисление точности
    accuracy_zero = accuracy_score([classes[label] for label in true_labels], zero_shot_preds)
    accuracy_one = accuracy_score([classes[label] for label in true_labels], one_shot_preds)
    accuracy_few = accuracy_score([classes[label] for label in true_labels], few_shot_preds)
    accuracy_prob = accuracy_score([classes[label] for label in true_labels], prob_preds)

    print(f"Zero-shot точность: {accuracy_zero:.2f}")
    print(f"One-shot точность: {accuracy_one:.2f}")
    print(f"Few-shot точность: {accuracy_few:.2f}")
    print(f"Вероятностная точность: {accuracy_prob:.2f}")

# Запуск оценки
evaluate_classifiers()


Текст 1: Футбольный матч между сборными России и Германии завершился со счётом 2:1 в пользу хозяев.
True Label: sport
Zero-shot:
Предсказанные значения: {'sport': 1, 'technology': 0, 'culinary': 0, 'politics': 0, 'health': 0}
One-shot:
Предсказанные значения: {'sport': 1, 'technology': 0, 'culinary': 0, 'politics': 0, 'health': 0}
Few-shot:
Предсказанные значения: {'sport': 0, 'technology': 1, 'culinary': 0, 'politics': 0, 'health': 1}
Начинаем вероятностную классификацию для текста: Футбольный матч между сборными...
Итерация 1, температура: 0.5
Предсказанные значения: {'sport': 1, 'technology': 0, 'culinary': 0, 'politics': 0, 'health': 0}
Итерация 2, температура: 0.7
Предсказанные значения: {'sport': 1, 'technology': 0, 'culinary': 0, 'politics': 0, 'health': 0}
Итерация 3, температура: 0.7
Предсказанные значения: {'sport': 1, 'technology': 0, 'culinary': 0, 'politics': 0, 'health': 0}
Итерация 4, температура: 1.2
Предсказанные значения: {'sport': 1, 'technology': 0, 'culinary': 0, 

Выводы:

Модель часто выдает либо ни одной единицы либо несколько, что указывает на неполное понимание задачи

Zero-shot и one-shot показали одинаковую точность (60%) , что указывает на схожую эффективность при отсутствии или наличии одного примера. Однако оба метода допускают ошибки, например, текст 4 (политика) классифицировался плохо всеми методами

Few-shot дал наихудший результат (40%) , вероятно из-за противоречивых примеров (например, текст 4 и 5 получили множественные неправильные метки). Это демонстрирует чувствительность метода к качеству примеров 

Вероятностный подход с усреднением сохранил точность 60%, но позволил учитывать неопределенность модели. Например, текст 5 получил вероятности [sport: 0.6, health: 0.4], что отражает связь физической активности с обоими классами 